In [2]:
import itertools
from gurobipy import Model, GRB

def solve_linear_program():
    model = Model("Linear_Programming_Example")
    
    attributes = ["x", "y", "z"]
    variables = {}
    for i in range(1, len(attributes) + 1):
        for subset in itertools.combinations(attributes, i):
            name = "".join(subset)
            name = "h(" + name + ")"
            # ub=len(subset)
            x = model.addVar(lb=0, name=name)
            variables[frozenset(subset)] = x
    
    model.setObjective(variables[frozenset(attributes)], GRB.MAXIMIZE)

    # Add constraints
    relations = [["x", "y"], ["y", "z"], ["z", "x"]]
    #relations = [["x"], ["y"], ["z"]]
    for rel in relations:
        model.addConstr(variables[frozenset(rel)] <= 1)
        
    # Trivial functional dependencies in query
    if False:
        query_head = relations + [attributes]
        print(query_head)
        for rel in query_head:
            for size in range(2, len(rel) + 1):
                for supset in itertools.combinations(rel, size):
                    # Loop over all subsets of the supset
                    for e in supset:
                        subset = [x for x in supset if x != e]
                        print(supset, subset, e)
                        if len(subset) > 0:
                            model.addConstr(variables[frozenset(subset)] == variables[frozenset(supset)])
        
    fds = [[["x", "z"], ["u"]], [["y", "u"], ["x"]], [["x", "z", "y"], ["u", "y"]], [["y", "u", "z"], ["x", "z"]]]
    #fds = [[["x"], ["z"]], [["y"], ["x"]], [["z"], ["y"]]]
    for Y, x in fds:
        Yx = Y + x
        #model.addConstr(variables[frozenset(Yx)] - variables[frozenset(Y)] == 0)
        
    # Shannon inequalities
    if True:
        # Monotonicity
        for x in attributes:
            Y = [y for y in attributes if y != x]
            model.addConstr(variables[frozenset(attributes)] - variables[frozenset(Y)] >= 0)
        
        # Submodularity
        for j, l in itertools.combinations(attributes, 2):
            S = [x for x in attributes if x != j and x != l]
            for i in range(1, len(S) + 1):
                for subset in itertools.combinations(S, i):
                    subset = list(subset)
                    jS = subset + [j]
                    lS = subset + [l]
                    jlS = subset + [j, l]
                    model.addConstr(variables[frozenset(jS)] + variables[frozenset(lS)] - variables[frozenset(subset)] - variables[frozenset(jlS)] >= 0)

    # Save model to lp file
    model.write("Linear_Programming_Example2.lp")

    # Optimize the model
    model.optimize()

    # Check optimization status
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found!")
        print(f"Objective value = {model.objVal}")
    elif model.status == GRB.INFEASIBLE:
        print("The model is infeasible.")
    elif model.status == GRB.UNBOUNDED:
        print("The model is unbounded.")
    else:
        print(f"Optimization ended with status {model.status}")

# Call the function
solve_linear_program()


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 9 rows, 7 columns and 21 nonzeros
Model fingerprint: 0x8e97222f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 3 columns
Presolve time: 0.00s
Presolved: 5 rows, 4 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0020000e+00   4.006000e+00   0.000000e+00      0s
       2    2.0000000e+00   0.000000e+00   0.000000e+00      0s

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 9 rows, 7 columns and 21 nonze